# Full-circle Atomistic Comparison of ETS

## Experimental data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import facets_drivers

%store -r notebook_data
globals().update(notebook_data)

In [ ]:
local_run = False

In [ ]:
from ewoksdask import execute_graph
if not local_run:
    from ewoksdask.schedulers import slurm_scheduler
    cluster = slurm_scheduler(account="staff-id01",queue="gpu", walltime="01:00:00", memory="256GB", cores=num_cores, job_extra_directives=["--gres=gpu:1"])

In [ ]:
from ewoksdask import execute_graph

nodes = [
    {
        "id": "read_positions",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.ReadLammpsPositions",
    },
    {
        "id": "display_read_figures",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.DisplayImages",
    },
    #{
    #    "id": "initial_scattering",
    #    "task_type": "class",
    #    "task_identifier": "facets_drivers.facets.InitialScattering",
    #},
    #{
    #    "id": "display_initial_figures",
    #    "task_type": "class",
    #    "task_identifier": "facets_drivers.facets.DisplayImages",
    #},
    {
        "id": "refine_grid",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.RefineGrid",
    },
    {
        "id": "display_refine_figures",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.DisplayImages",
    },
    {
        "id": "post_process",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.PostProcessScattering",
    },
    {
        "id": "display_postprocess_figures",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.DisplayImages",
    },
]

links = [
    # Read positions -> Initial Scattering
    #{
    #    "source": "read_positions",
    #    "target": "initial_scattering",
    #    "data_mapping": [{"source_output": "positions", "target_input": "positions"}],
    #},
    # Read positions -> Refine Grid
    {
        "source": "read_positions",
        "target": "refine_grid",
        "data_mapping": [{"source_output": "positions", "target_input": "positions"}],
    },
    # Display figures from each main task
    {
        "source": "read_positions",
        "target": "display_read_figures",
        "data_mapping": [{"source_output": "figures", "target_input": "image_data_list"}],
    },
    #{
    #    "source": "initial_scattering",
    #    "target": "display_initial_figures",
    #    "data_mapping": [{"source_output": "figures", "target_input": "image_data_list"}],
    #},
    {
        "source": "refine_grid",
        "target": "display_refine_figures",
        "data_mapping": [{"source_output": "figures", "target_input": "image_data_list"}],
    },
    # Refine Grid -> Post Process
    {
        "source": "refine_grid",
        "target": "post_process",
        "data_mapping": [
            {"source_output": "refined_q_grid", "target_input": "refined_q_grid"},
            {"source_output": "scattered_amp", "target_input": "scattered_amp"},
        ],
    },
    # Display figures from Post Process
    {
        "source": "post_process",
        "target": "display_postprocess_figures",
        "data_mapping": [{"source_output": "figures", "target_input": "image_data_list"}],
    },
]

# Combine nodes and links into the final workflow structure
workflow = {"graph": {"id": "simulation_analysis"}, "nodes": nodes, "links": links}

inputs = [
    # Inputs for ReadLammpsPositions
    {"id": "read_positions", "name": "file_path", "value": f"{simulation_directory}/exp_minimized.xyz"},
    {"id": "read_positions", "name": "centre", "value": True},

    # Inputs for InitialScattering
    #{"id": "initial_scattering", "name": "hkl", "value": hkl},
    #{"id": "initial_scattering", "name": "lattice_parameter", "value": 3.92},
    #{"id": "initial_scattering", "name": "q_size", "value": (200, 200, 200)},
    #{"id": "initial_scattering", "name": "step_nb", "value": 400},

    # Inputs for RefineGrid
    {"id": "refine_grid", "name": "hkl", "value": hkl},
    {"id": "refine_grid", "name": "initial_lattice_guess", "value": 3.92e-10},
    {"id": "refine_grid", "name": "q_size", "value": (200, 200, 200)},
    {"id": "refine_grid", "name": "step_nb", "value": 1600},
    {"id": "refine_grid", "name": "max_iterations", "value": 10},

    # Input for PostProcessScattering
    {"id": "post_process", "name": "hkl", "value": hkl},
]
# issue with initial_scattering on remote cluster. no idea why. positions.copy() does not solve the issue
# race condition between the two pynx runs?

In [ ]:
%%time
result = execute_graph(
    workflow,
    inputs=inputs,
    scheduler=None if local_run else cluster.scheduler_address,
    outputs=[
        {'id': 'read_positions'}, 
        {'id': 'initial_scattering'},
        {'id': 'refine_grid'},
        {'id': 'post_process'},
    ],
    merge_outputs=False,
)

In [ ]:
from IPython.display import display, Image
[[display(Image(data=figure)) for figure in result[current]["figures"]] for current in result]

In [ ]:
if not local_run:
    cluster.close()